In [1]:
import requests
from bs4 import BeautifulSoup as bs
r = requests.get('https://www.onetonline.org/find/family?f=15&g=Go')
webpage = bs(r.content)

In [2]:
job_links = []
for i in webpage.find('table').find_all('tr'):
    job_links.append(i.a['href'])
len(job_links)
job_links = job_links[1:]
job_links = job_links[:9] + job_links[10:]
job_links = job_links[:24] + job_links[25:]
len(job_links)

36

In [3]:
occupation = []
for i in webpage.find('table').find_all('tr')[1:]:
    occupation.append(i.a.text.lower())
occupation.remove('Computer Occupations, All Other'.lower())
occupation.remove('Mathematical Science Occupations, All Other'.lower())
len(occupation)

36

In [5]:
jlist = []
for id_, link in enumerate(job_links):
    ra = requests.get(link)
    bsa = bs(ra.content)
    skll = bsa.body.select('div.section_TechnologySkills') 
    if skll:
        x = [occupation[id_].lower()]
        for i in skll[1].ul.find_all('li'):
            x.append(i.b.text.lower())
        jlist.append(x.copy())
len(jlist)

31

In [6]:
import gensim
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [7]:
model = gensim.models.Word2Vec(window = 10,
                              min_count = 1,
                              workers = 2)

In [8]:
model.build_vocab(jlist, progress_per = 1000)

In [9]:
model.train(jlist, total_examples = model.corpus_count, epochs = model.epochs)

(2463, 6595)

In [10]:
model.wv.most_similar('Clinical Data Managers'.lower())

[('gateway software', 0.22438864409923553),
 ('analytical or scientific software', 0.21388930082321167),
 ('bioinformatics technicians', 0.20640847086906433),
 ('customer relationship management crm software', 0.1909753829240799),
 ('map creation software', 0.18360300362110138),
 ('computer based training software', 0.18159446120262146),
 ('business intelligence and data analysis software', 0.17151044309139252),
 ('bridge software', 0.16743925213813782),
 ('configuration management software', 0.16414767503738403),
 ('actuaries', 0.15803739428520203)]

### word_encoding

In [11]:
skills = [i for j in jlist
         for i in j[1:]]

In [12]:
df = pd.DataFrame(columns = ['skills']+occupation)
df.head()

,skills,actuaries,bioinformatics technicians,biostatisticians,blockchain engineers,business intelligence analysts,clinical data managers,computer and information research scientists,computer network architects,computer network support specialists,...,operations research analysts,penetration testers,software developers,software quality assurance analysts and testers,statisticians,telecommunications engineering specialists,video game designers,web administrators,web and digital interface designers,web developers


In [13]:
skills = pd.DataFrame(columns = ['s'], data = skills)
skills.head()

,s
0,analytical or scientific software
1,business intelligence and data analysis software
2,compliance software
3,data base user interface and query software
4,development environment software


In [14]:
df = pd.DataFrame(columns = ['skills'] + occupation)

In [15]:
df['skills'] = skills.s.unique() # to get unique skills

In [16]:
i_to_s = {}
s_to_i = {}
for i,val in enumerate(skills.s.unique()):
    i_to_s[i] = val # retreivel
    s_to_i[val] = i  # inserting

In [17]:
t = len(skills.s.unique())
data = [] # vectors for jobs 
col_list = []
for i in jlist:
    x = [0] * t
    col_list.append(i[0])
    for j in i[1:]:
        x[s_to_i[j]] = 1
    data.append(x)
data[0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [24]:
#col_list = [j[0] for j in jlist] # only col with tech_skills 
len(col_list)

31

In [18]:
for i,val in enumerate(col_list):
    df[val] = data[i]
df['data scientists'] = df['business intelligence analysts']

In [19]:
df.index = df.skills

In [20]:
df.drop('skills', inplace = True, axis = 1)

In [21]:
[i for i in occupation if not(i in col_list)] # jobs without tech_skills

['blockchain engineers',
 'data scientists',
 'digital forensics analysts',
 'information security engineers',
 'penetration testers']

In [22]:
# measuring similarity
import scipy.spatial

In [56]:
# juccard dist/index formula
inter = sum(df['actuaries'] * df['business intelligence analysts'])
uni = sum(df['actuaries']) + sum(df['business intelligence analysts'] )- inter
1 - inter / uni

0.7254901960784313

In [24]:
jaccard = scipy.spatial.distance.cdist(df.T, df.T, metric='jaccard')
df_job_sim = pd.DataFrame(jaccard, columns = occupation, index = occupation)

In [25]:
df_job_sim.head()

,actuaries,bioinformatics technicians,biostatisticians,blockchain engineers,business intelligence analysts,clinical data managers,computer and information research scientists,computer network architects,computer network support specialists,computer programmers,...,operations research analysts,penetration testers,software developers,software quality assurance analysts and testers,statisticians,telecommunications engineering specialists,video game designers,web administrators,web and digital interface designers,web developers
actuaries,0.000000,0.576923,0.608696,1.0,0.725490,0.523810,0.702703,0.781250,0.767442,0.730769,...,0.628571,1.0,0.758621,0.784615,0.500000,0.666667,0.541667,0.657895,0.720000,0.725490
bioinformatics technicians,0.576923,0.000000,0.375000,1.0,0.615385,0.480000,0.589744,0.671875,0.571429,0.648148,...,0.513514,1.0,0.661017,0.656250,0.520000,0.575758,0.444444,0.585366,0.660377,0.641509
biostatisticians,0.608696,0.375000,0.000000,1.0,0.660000,0.428571,0.571429,0.769231,0.690476,0.666667,...,0.648649,1.0,0.724138,0.734375,0.315789,0.727273,0.458333,0.675000,0.680000,0.660000
blockchain engineers,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
business intelligence analysts,0.725490,0.615385,0.660000,1.0,0.000000,0.705882,0.433962,0.452055,0.533333,0.196429,...,0.340000,1.0,0.301587,0.246154,0.700000,0.660714,0.653846,0.491228,0.293103,0.245614


In [26]:
jobs_closer = {} # dictionary of jobs with relavent jobs as ranked list
for i in occupation:
    jobs = df_job_sim[i].nsmallest(10)
    jdata = {i : [j for j in jobs.index if j != i]}
    jobs_closer.update(jdata)

In [30]:
df.to_csv('jobs_skills.csv')
df_job_sim.to_csv('job_similar.csv')

#### similar skills

In [31]:
skills_list = []
for id_, link in enumerate(job_links):
    ra = requests.get(link)
    bsa = bs(ra.content)
    skll = bsa.body.select('div.section_TechnologySkills') 
    if skll:
        for i in skll[1].ul.find_all('li'):
            x = []
            x.append(i.b.text.lower()) # tech_skill
            x.extend([j.strip() for j in i.text.lower().strip().split('(')[0].split('—')[-1].split(';')])  # all ex
            skills_list.append(x)
skills_list[0]

['analytical or scientific software',
 'ibm spss statistics',
 'insightful s-plus',
 'sas',
 'statistical software']

In [32]:
exampls_list = [ i.strip() for j in skills_list for i in j[1:]]
len(exampls_list)

2896

In [33]:
exampls_list

['ibm spss statistics',
 'insightful s-plus',
 'sas',
 'statistical software',
 'qlik tech qlikview',
 'compliance testing software',
 'data entry software',
 'microsoft sql server',
 'oracle software',
 'structured query language sql',
 'microsoft visual basic',
 'microsoft visual basic for applications vba',
 'ibm lotus notes',
 'ggy axis',
 'oak mountain software annuityvalue',
 'pricing software',
 'towers perrin moses',
 'microsoft visio',
 'c++',
 'oracle java',
 'python',
 'r',
 'microsoft visual foxpro',
 'microsoft office',
 'microsoft powerpoint',
 'microsoft project',
 'microsoft excel',
 'microsoft word',
 'avaya identity engines',
 'data visualization software',
 'ibm spss statistics',
 'sas',
 'the mathworks matlab',
 'customer relationship management crm software',
 'microsoft sql server',
 'microsoft access',
 'mysql',
 'oracle software',
 'structured query language sql',
 'c',
 'ruby',
 'microsoft outlook',
 'atlassian bamboo',
 'jenkins ci',
 'sap software',
 'apache 

In [34]:
df_ex = pd.DataFrame(columns = ['example'], data = exampls_list)

In [35]:
len(df_ex['example'].unique())

656

In [36]:
i_to_ex = {}
ex_to_i = {}
for i,val in enumerate(df_ex.example.unique()):
    i_to_ex[i] = val # retreivel
    ex_to_i[val] = i  # inserting

In [37]:
t = len(df_ex.example.unique())
data = [] # vectors for skills  
col = []
for i in skills_list:
    x = [0] * t
    col.append(i[0])
    for j in i[1:]:
        x[ex_to_i[j]] = 1
    data.append(x)
data[0]

[1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [38]:
len(data), len(skills.s)

(1288, 1288)

In [39]:
df2 = pd.DataFrame(columns = skills.s.unique(), index = df_ex.example.unique())

In [40]:
df2.head()

,analytical or scientific software,business intelligence and data analysis software,compliance software,data base user interface and query software,development environment software,electronic mail software,financial analysis software,graphics or photo imaging software,object or component oriented development software,object oriented data base management software,...,network operation system software,manufacturing execution system mes software,storage media loading software,data compression software,optical character reader ocr or scanning software,mobile location based services software,point of sale pos software,gateway software,device drivers or system software,music or sound editing software
ibm spss statistics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
insightful s-plus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
statistical software,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
qlik tech qlikview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
for i,val in enumerate(data):
    temp = skills['s'][i]
    if pd.isnull(df2[temp][0]):
        df2[temp] = val
    else:
        df2[temp] = [max(df2[temp][j], val[j]) for j in range(len(val))]


In [42]:
df2.head()

,analytical or scientific software,business intelligence and data analysis software,compliance software,data base user interface and query software,development environment software,electronic mail software,financial analysis software,graphics or photo imaging software,object or component oriented development software,object oriented data base management software,...,network operation system software,manufacturing execution system mes software,storage media loading software,data compression software,optical character reader ocr or scanning software,mobile location based services software,point of sale pos software,gateway software,device drivers or system software,music or sound editing software
ibm spss statistics,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
insightful s-plus,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sas,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
statistical software,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
qlik tech qlikview,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
jaccard2 = scipy.spatial.distance.cdist(df2.T, df2.T, metric='jaccard')
df_skl_sim = pd.DataFrame(jaccard2, columns = skills.s.unique(), index = skills.s.unique())

In [44]:
df_skl_sim.head()

,analytical or scientific software,business intelligence and data analysis software,compliance software,data base user interface and query software,development environment software,electronic mail software,financial analysis software,graphics or photo imaging software,object or component oriented development software,object oriented data base management software,...,network operation system software,manufacturing execution system mes software,storage media loading software,data compression software,optical character reader ocr or scanning software,mobile location based services software,point of sale pos software,gateway software,device drivers or system software,music or sound editing software
analytical or scientific software,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
business intelligence and data analysis software,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.958333,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
compliance software,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
data base user interface and query software,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.000000,0.96,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
development environment software,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.975610,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [45]:
skills_closer = {} # dictionary of jobs with relavent jobs as ranked list
for i in skills.s.unique():
    t1 = df_skl_sim[i].nsmallest(7)
    sdt = {i : [j for j in t1.index if j != i]}
    skills_closer.update(sdt)

In [46]:
df_skl_sim['data base user interface and query software'][skills_closer['data base user interface and query software']]

data base management system software                0.903226
object oriented data base management software       0.960000
computer based training software                    0.961538
analytical or scientific software                   1.000000
business intelligence and data analysis software    1.000000
compliance software                                 1.000000
Name: data base user interface and query software, dtype: float64

In [47]:
df2.to_csv('skills_exampls.csv')

### input_function

In [55]:
[1,2,3] * [3,4,1]

TypeError: can't multiply sequence by non-int of type 'list'

In [74]:
def jacard_dist(v1, v2):
    intr = sum([v1[i] * v2[i] for i in range(len(v1))])
    uni = sum(v1) + sum(v2) - intr
    return intr/uni

In [86]:
def find(input_list, job = None):
    indx1 = df.index
    indx2 = df2.index
    s = [0] * len(indx1)
    e = [0] * len(indx2)
    input_list = [i.lower() for i in input_list]
    for i in input_list:
        if i in indx1: # searching skills
            s[s_to_i[i]] = 1
        elif i in indx2:
            e[ex_to_i[i]] = 1
            for j in df2.columns:
                if df2.loc[i][j]:
                    s[s_to_i[j]] = 1
    print(sum(e),sum(s),len(input_list))
    if sum(s) > 0:
        min_ = 1
        j = job
        for i in df.columns:
            dist = jacard_dist(s,df[i])
            #j = i if dist < min_ else job
            if dist < min_:
                j = i
            min_ = min(min_, dist)
        print(j)
    
    return [j,jobs_closer[j]]

In [54]:
len(df2.index)

656

In [81]:
job = 'business intelligence analysts'
skills = ['Access software', 'GitHub','Oracle Business Intelligence Enterprise Edition','Electronic mail software',
         'MongoDB','Microsoft Excel', 'Microsoft Visual Basic','python','c++','c', 'sql']

In [82]:
find(skills)

8 8 11
computer user support specialists None


['software quality assurance analysts and testers',
 'computer systems analysts',
 'computer systems engineers/architects',
 'database administrators',
 'information technology project managers',
 'network and computer systems administrators',
 'software developers',
 'information security analysts',
 'web developers']

In [61]:
'S'.lower()

's'

In [85]:
df_job_sim['computer user support specialists'][['software quality assurance analysts and testers',
 'computer systems analysts',
 'computer systems engineers/architects',
 'database administrators',
 'information technology project managers',
 'network and computer systems administrators',
 'software developers',
 'information security analysts',
 'web developers']]

software quality assurance analysts and testers    0.152778
computer systems analysts                          0.164384
computer systems engineers/architects              0.178082
database administrators                            0.219178
information technology project managers            0.236111
network and computer systems administrators        0.240000
software developers                                0.250000
information security analysts                      0.253521
web developers                                     0.300000
Name: computer user support specialists, dtype: float64